In [1]:
import pandas as pd
from pathlib import Path

project = Path.cwd()
data_raw = project.parent / "data_raw"

file_2015 = data_raw / "amazon_india_2015.csv"
df = pd.read_csv(file_2015, low_memory=False)

df.shape


(33165, 34)

In [2]:
# CLEANING 1: Clean order_date

def clean_order_date(col):
    # Convert to datetime
    # errors='coerce' converts bad dates to NaT
    # dayfirst=True handles DD/MM/YYYY formats
    col = pd.to_datetime(col, errors='coerce', dayfirst=True)
    return col

df['order_date_clean'] = clean_order_date(df['order_date'])

# Check how many invalid dates became NaT (missing)
df['order_date_clean'].isna().sum()


C:\Users\DELL\AppData\Local\Temp\ipykernel_17288\829849314.py:7: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  col = pd.to_datetime(col, errors='coerce', dayfirst=True)


np.int64(3252)

In [3]:
# show sample raw date strings which produced NaT
bad_mask = df['order_date_clean'].isna()
bad_dates = df.loc[bad_mask, 'order_date'].astype(str)

print("Total bad dates:", bad_dates.shape[0])
print("\nTop 30 most common raw strings among bad dates:")
print(bad_dates.value_counts().head(30))


Total bad dates: 3252

Top 30 most common raw strings among bad dates:
order_date
22/12/2015    12
10/07/2015    12
12/05/2015    11
10/08/2015    11
03/07/2015    11
24-12-2015    11
13/10/2015    11
12/10/2015    11
03/12/2015    10
09/05/2015    10
07/09/2015    10
01/04/2015    10
12/06/2015     9
14/10/2015     9
13/09/2015     9
11/04/2015     9
12/12/2015     9
10/01/2015     9
25-07-2015     9
05/10/2015     9
06/09/2015     9
10/11/2015     9
21/10/2015     9
02/09/2015     9
10-11-2015     9
11/23/2015     9
29-04-2015     9
07/06/2015     8
01/03/2015     8
31-08-2015     8
Name: count, dtype: int64


In [4]:
# attempt fallback parsing for remaining NaT rows
mask_na = df['order_date_clean'].isna()
print("Rows to attempt fallback parse:", mask_na.sum())

# Try parsing with dayfirst=False first
df.loc[mask_na, 'order_date_clean2'] = pd.to_datetime(df.loc[mask_na, 'order_date'],
                                                       errors='coerce',
                                                       dayfirst=False,
                                                       infer_datetime_format=True,
                                                       yearfirst=False)

recovered1 = df['order_date_clean2'].notna().sum()
print("Recovered with dayfirst=False:", int(recovered1))

# For any still missing, try yearfirst=True (handles YYYY/MM/DD)
mask_still = df['order_date_clean2'].isna() & df['order_date_clean'].isna()
df.loc[mask_still, 'order_date_clean2'] = pd.to_datetime(df.loc[mask_still, 'order_date'],
                                                         errors='coerce',
                                                         infer_datetime_format=True,
                                                         yearfirst=True)

recovered2 = df['order_date_clean2'].notna().sum() - recovered1
print("Recovered with yearfirst=True:", int(recovered2))

# Final counts
total_recovered = df['order_date_clean2'].notna().sum()
still_missing = df['order_date_clean'].isna().sum() - total_recovered
print("Total recovered in order_date_clean2:", int(total_recovered))
print("Remaining missing after fallback attempts:", int(still_missing))

# If order_date_clean2 has values, fill original clean column
df['order_date_clean'] = df['order_date_clean'].combine_first(df['order_date_clean2'])

print("Final missing after combining:", int(df['order_date_clean'].isna().sum()))


Rows to attempt fallback parse: 3252
Recovered with dayfirst=False: 1307
Recovered with yearfirst=True: 1192
Total recovered in order_date_clean2: 2499
Remaining missing after fallback attempts: 753
Final missing after combining: 753


C:\Users\DELL\AppData\Local\Temp\ipykernel_17288\676345187.py:6: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df.loc[mask_na, 'order_date_clean2'] = pd.to_datetime(df.loc[mask_na, 'order_date'],
C:\Users\DELL\AppData\Local\Temp\ipykernel_17288\676345187.py:17: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df.loc[mask_still, 'order_date_clean2'] = pd.to_datetime(df.loc[mask_still, 'order_date'],
C:\Users\DELL\AppData\Local\Temp\ipykernel_17288\676345187.py:17: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pa

In [5]:
# A: show top raw strings that remain missing after previous attempts
bad_mask = df['order_date_clean'].isna()
bad_dates = df.loc[bad_mask, 'order_date'].astype(str)

print("Total still-missing rows:", bad_dates.shape[0])
print("\nTop 40 most common raw strings among remaining bad dates:\n")
print(bad_dates.value_counts().head(40))


Total still-missing rows: 753

Top 40 most common raw strings among remaining bad dates:

order_date
22/12/2015    12
13/10/2015    11
13/09/2015     9
21/10/2015     9
14/10/2015     9
22/10/2015     8
27/04/2015     7
30/07/2015     7
26/09/2015     7
26/05/2015     7
20/12/2015     7
16/12/2015     7
26/10/2015     7
26/06/2015     6
17/09/2015     6
27/10/2015     6
21/11/2015     6
17/07/2015     6
29/01/2015     6
25/03/2015     6
21/03/2015     6
20/05/2015     6
19/09/2015     6
26/12/2015     6
24/11/2015     6
25/12/2015     6
27/12/2015     6
23/10/2015     6
28/11/2015     5
23/11/2015     5
23/07/2015     5
21/06/2015     5
15/06/2015     5
16/07/2015     5
17/06/2015     5
26/08/2015     5
15/07/2015     5
19/07/2015     5
14/01/2015     5
27/02/2015     5
Name: count, dtype: int64


In [6]:
# B: aggressive fallback using regex extraction + dateutil
import re
from dateutil import parser

def extract_date_like(s):
    if pd.isna(s):
        return None
    s = str(s).strip()
    # remove common noisy words
    s_clean = re.sub(r'(?i)\b(approx|approx\.|around|est|estimated|tbd|unknown|na|n/a|not available)\b', ' ', s)
    s_clean = s_clean.replace(',', ' ').replace(';', ' ').replace('.', ' ')
    # find first date-like substring (several common patterns)
    patterns = [
        r'\d{1,2}[-/]\d{1,2}[-/]\d{2,4}',    # 12-05-2019 or 12/05/2019
        r'\d{4}[-/]\d{1,2}[-/]\d{1,2}',      # 2019-05-12 or 2019/5/12
        r'\d{1,2}\s+[A-Za-z]{3,9}\s+\d{4}',  # 12 Jan 2019
        r'[A-Za-z]{3,9}\s+\d{1,2},?\s+\d{4}',# Jan 12 2019 or January 12, 2019
        r'\d{4}'                             # year only (will parse to 1 Jan of that year)
    ]
    for pat in patterns:
        m = re.search(pat, s_clean)
        if m:
            return m.group(0)
    return s_clean  # as last resort, try the whole cleaned string

def try_parse_date(s):
    if s is None or s.strip()=="":
        return None
    try:
        # parser.parse with dayfirst=True to prefer D/M/Y where ambiguous
        dt = parser.parse(s, dayfirst=True, fuzzy=True)
        return pd.to_datetime(dt)
    except Exception:
        return None

# Apply only to still-missing rows
mask = df['order_date_clean'].isna()
candidates = df.loc[mask, 'order_date'].astype(str)

# extract then parse
extracted = candidates.map(extract_date_like)
parsed = extracted.map(try_parse_date)

# How many parsed?
num_parsed = parsed.notna().sum()
print("Parsed by regex+dateutil:", int(num_parsed))

# Fill into order_date_clean where parsed success
df.loc[mask, 'order_date_clean'] = df.loc[mask, 'order_date_clean'].combine_first(parsed)

print("Final missing after regex+dateutil attempt:", int(df['order_date_clean'].isna().sum()))


Parsed by regex+dateutil: 753
Final missing after regex+dateutil attempt: 0


In [7]:
# CLEANING 2: Clean original_price_inr

def clean_price(x):
    if pd.isna(x):
        return None
    
    x = str(x).strip()
    
    # Remove currency symbols and commas
    x = x.replace("₹", "").replace(",", "").strip()
    
    # Handle non-numeric text cases
    bad_values = ["price on request", "not available", "na", "n/a", "-", "", "unknown"]
    if x.lower() in bad_values:
        return None

    # Try converting to float
    try:
        return float(x)
    except:
        return None

df['original_price_clean'] = df['original_price_inr'].apply(clean_price)

df['original_price_clean'].head(10)


0    123614.29
1     54731.86
2     97644.25
3     21947.26
4     54731.86
5    131194.65
6     86987.64
7     32169.01
8     40264.16
9     54731.86
Name: original_price_clean, dtype: float64

In [8]:
df['original_price_clean'].isna().sum()


np.int64(987)

In [9]:
# CLEANING 3: Boolean columns (Yes/No/Y/N/1/0 → True/False)

def clean_bool(x):
    if pd.isna(x):
        return False
    x = str(x).strip().lower()
    return x in ["yes", "y", "1", "true"]

bool_columns = ['is_prime_member', 'is_festival_sale', 'is_prime_eligible']

for col in bool_columns:
    df[col + "_clean"] = df[col].apply(clean_bool)

df[[col + "_clean" for col in bool_columns]].head()


,is_prime_member_clean,is_festival_sale_clean,is_prime_eligible_clean
0,False,True,True
1,False,False,True
2,False,True,True
3,False,False,True
4,False,False,True


In [10]:
# CLEANING 4: customer_rating normalization

import re
import numpy as np

def clean_rating_val(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip().lower()
    # common non-rating markers
    if s in ["na", "n/a", "nr", "no rating", "-", "none", "unknown", "not rated"]:
        return np.nan

    # "4 stars" -> 4
    m = re.match(r'^(\d+(\.\d+)?)\s*stars?$', s)
    if m:
        try:
            return float(m.group(1))
        except:
            return np.nan

    # "3/5" or "2.5/5.0" -> take numerator
    if '/' in s:
        parts = s.split('/')
        try:
            val = float(parts[0])
            return val
        except:
            # maybe string like "Rated 4 out of 5"
            pass

    # strings like "rated 4 out of 5"
    m = re.search(r'(\d+(\.\d+)?)\s*(?:out of|/)\s*\d+', s)
    if m:
        try:
            return float(m.group(1))
        except:
            return np.nan

    # plain numeric
    try:
        return float(s)
    except:
        return np.nan

# Apply to customer_rating
df['customer_rating_clean'] = df['customer_rating'].apply(clean_rating_val)

# Quick checks
total = len(df)
parsed = df['customer_rating_clean'].notna().sum()
missing_after = df['customer_rating_clean'].isna().sum()

print(f"Total rows examined: {total}")
print(f"Parsed numeric ratings: {parsed}")
print(f"Missing after cleaning: {missing_after}")

# show a few examples side-by-side for manual inspection
df[['customer_rating', 'customer_rating_clean']].drop_duplicates().head(15)


Total rows examined: 33165
Parsed numeric ratings: 23196
Missing after cleaning: 9969


,customer_rating,customer_rating_clean
0,5.0,5.0
1,4.5,4.5
2,NaN,NaN
3,3.0,3.0
4,4.0,4.0
6,3.5,3.5
13,4/5,4.0
16,5.0 stars,5.0
62,5.0/5.0,5.0
67,4.5 stars,4.5


In [11]:
# CLEANING 5: Normalize customer_city

def clean_city(x):
    if pd.isna(x):
        return None
    x = str(x).strip().lower()
    
    # Replace slashes with space
    x = x.replace("/", " ")
    
    # Fix common city variants
    replacements = {
        "bengaluru": "bangalore",
        "bengaluru bangalore": "bangalore",
        "bangalore bengaluru": "bangalore",
        "bombay": "mumbai",
        "new delhi": "delhi",
    }
    
    # Direct replacement if exact match
    if x in replacements:
        return replacements[x]
    
    # Partial replacement
    for old, new in replacements.items():
        if old in x:
            return new
    
    # Remove multiple spaces
    x = " ".join(x.split())
    
    return x

df['customer_city_clean'] = df['customer_city'].apply(clean_city)

df[['customer_city', 'customer_city_clean']].head(20)
# CLEANING 5: Normalize customer_city

def clean_city(x):
    if pd.isna(x):
        return None
    x = str(x).strip().lower()
    
    # Replace slashes with space
    x = x.replace("/", " ")
    
    # Fix common city variants
    replacements = {
        "bengaluru": "bangalore",
        "bengaluru bangalore": "bangalore",
        "bangalore bengaluru": "bangalore",
        "bombay": "mumbai",
        "new delhi": "delhi",
    }
    
    # Direct replacement if exact match
    if x in replacements:
        return replacements[x]
    
    # Partial replacement
    for old, new in replacements.items():
        if old in x:
            return new
    
    # Remove multiple spaces
    x = " ".join(x.split())
    
    return x

df['customer_city_clean'] = df['customer_city'].apply(clean_city)

df[['customer_city', 'customer_city_clean']].head(20)


,customer_city,customer_city_clean
0,Mumbai,mumbai
1,Allahabad,allahabad
2,Mumbai,mumbai
3,Kolkata,kolkata
4,Ludhiana,ludhiana
5,Delhi,delhi
6,Lucknow,lucknow
7,Jaipur,jaipur
8,Delhi,delhi
9,Bhubaneswar,bhubaneswar


In [12]:
# A1 - quick info
print("shape:", df.shape)
print("\nData types:")
print(df.dtypes.value_counts())

# show nulls in important cleaned columns
cols_check = [
  'order_date_clean','original_price_clean',
  'is_prime_member_clean','is_festival_sale_clean','is_prime_eligible_clean',
  'customer_rating_clean','customer_city_clean'
]
print("\nMissing counts for key cleaned cols:")
print(df[cols_check].isna().sum())
# A1 - quick info
print("shape:", df.shape)
print("\nData types:")
print(df.dtypes.value_counts())

# show nulls in important cleaned columns
cols_check = [
  'order_date_clean','original_price_clean',
  'is_prime_member_clean','is_festival_sale_clean','is_prime_eligible_clean',
  'customer_rating_clean','customer_city_clean'
]
print("\nMissing counts for key cleaned cols:")
print(df[cols_check].isna().sum())


shape: (33165, 42)

Data types:
object            24
float64            9
int64              4
bool               3
datetime64[ns]     2
Name: count, dtype: int64

Missing counts for key cleaned cols:
order_date_clean              0
original_price_clean        987
is_prime_member_clean         0
is_festival_sale_clean        0
is_prime_eligible_clean       0
customer_rating_clean      9969
customer_city_clean           0
dtype: int64
shape: (33165, 42)

Data types:
object            24
float64            9
int64              4
bool               3
datetime64[ns]     2
Name: count, dtype: int64

Missing counts for key cleaned cols:
order_date_clean              0
original_price_clean        987
is_prime_member_clean         0
is_festival_sale_clean        0
is_prime_eligible_clean       0
customer_rating_clean      9969
customer_city_clean           0
dtype: int64


In [13]:
# A2 - sample rows to visually inspect
df[['order_date','order_date_clean','original_price_inr','original_price_clean',
    'is_prime_member','is_prime_member_clean','customer_rating','customer_rating_clean',
    'customer_city','customer_city_clean']].head(15)


,order_date,order_date_clean,original_price_inr,original_price_clean,is_prime_member,is_prime_member_clean,customer_rating,customer_rating_clean,customer_city,customer_city_clean
0,2015-01-25,2015-01-25,123614.29,123614.29,No,False,5.0,5.0,Mumbai,mumbai
1,2015-01-05,2015-01-05,54731.86,54731.86,False,False,4.5,4.5,Allahabad,allahabad
2,2015-01-24,2015-01-24,97644.25,97644.25,False,False,NaN,NaN,Mumbai,mumbai
3,2015-01-28,2015-01-28,"21,947.26",21947.26,False,False,3.0,3.0,Kolkata,kolkata
4,2015-01-31,2015-01-31,54731.86,54731.86,False,False,4.0,4.0,Ludhiana,ludhiana
5,2015-01-04,2015-01-04,131194.65,131194.65,False,False,4.5,4.5,Delhi,delhi
6,2015-01-27,2015-01-27,86987.64,86987.64,False,False,3.5,3.5,Lucknow,lucknow
7,2015-01-08,2015-01-08,32169.01,32169.01,False,False,4.0,4.0,Jaipur,jaipur
8,2015-01-18,2015-01-18,40264.16,40264.16,False,False,5.0,5.0,Delhi,delhi
9,2015-01-03,2015-01-03,54731.86,54731.86,False,False,4.5,4.5,Bhubaneswar,bhubaneswar


In [14]:
# A3 - date range
print("min/max order_date_clean:", df['order_date_clean'].min(), df['order_date_clean'].max())
print("year value counts (top):")
print(df['order_date_clean'].dt.year.value_counts().sort_index())


min/max order_date_clean: 2015-01-01 00:00:00 2015-12-31 00:00:00
year value counts (top):
order_date_clean
2015    33165
Name: count, dtype: int64


In [15]:
from pathlib import Path
out_dir = Path.cwd().parent / "data_cleaned"
out_dir.mkdir(exist_ok=True)
out_path = out_dir / "amazon_india_2015_clean_sample.csv"
df.to_csv(out_path, index=False)
print("Saved cleaned sample to:", out_path)


Saved cleaned sample to: c:\Users\DELL\Desktop\amazon_project\data_cleaned\amazon_india_2015_clean_sample.csv


In [16]:
# STEP 1: Clean category and subcategory

def clean_text(x):
    if pd.isna(x):
        return None
    s = str(x).lower().strip()
    s = s.replace("&", "and")          # unify electronics & accessories → electronics and accessories
    s = " ".join(s.split())            # remove double spaces
    return s

df['category_clean'] = df['category'].apply(clean_text)
df['subcategory_clean'] = df['subcategory'].apply(clean_text)

# Mapping known variants
category_map = {
    "electronics and accessories": "electronics",
    "electronic": "electronics",
    "electronics accessories": "electronics",
    "mobile phones": "mobiles",
    "mobile and accessories": "mobiles",
    "fashion men": "fashion",
    "fashion women": "fashion",
}

def map_category(s):
    if s in category_map:
        return category_map[s]
    # try partial match
    for k, v in category_map.items():
        if k in s:
            return v
    return s

df['category_clean'] = df['category_clean'].apply(map_category)

df[['category', 'category_clean', 'subcategory', 'subcategory_clean']].head(15)


,category,category_clean,subcategory,subcategory_clean
0,Electronics,electronics,Smartphones,smartphones
1,Electronics,electronics,Smartphones,smartphones
2,Electronics,electronics,Smartphones,smartphones
3,Electronics,electronics,Smartphones,smartphones
4,Electronics,electronics,Smartphones,smartphones
5,Electronics,electronics,Laptops,laptops
6,Electronics,electronics,Smartphones,smartphones
7,Electronics,electronics,Smartphones,smartphones
8,Electronics,electronics,Smartphones,smartphones
9,Electronics,electronics,Smartphones,smartphones


In [17]:
# STEP 2: Clean delivery_days → numeric

import re

def clean_delivery_days(x):
    if pd.isna(x):
        return None
    
    s = str(x).strip().lower()

    # same day handling
    if s in ["same day", "sameday", "0", "0 day", "0 days"]:
        return 0.0
    
    # range like "1-3 days" or "1 to 4 days"
    m_range = re.search(r'(\d{1,2})\s*[-to]+\s*(\d{1,2})', s)
    if m_range:
        a = int(m_range.group(1))
        b = int(m_range.group(2))
        return (a + b) / 2   # midpoint
    
    # direct number inside the string
    m_num = re.search(r'(\d{1,2})', s)
    if m_num:
        return float(m_num.group(1))
    
    return None

df['delivery_days_clean'] = df['delivery_days'].apply(clean_delivery_days)

df[['delivery_days', 'delivery_days_clean']].head(20)


,delivery_days,delivery_days_clean
0,6,6.0
1,4,4.0
2,4,4.0
3,4,4.0
4,3,3.0
5,3,3.0
6,4,4.0
7,3,3.0
8,3,3.0
9,5,5.0


In [18]:
# STEP 3: Clean payment_method → UPI / Card / COD / NetBanking

def clean_payment_method(x):
    if pd.isna(x):
        return None
    s = str(x).lower().replace(" ", "").replace("-", "")
    
    # UPI group
    if any(k in s for k in ['upi', 'gpay', 'googlepay', 'phonepe', 'paytm']):
        return 'UPI'
    
    # Card group
    if any(k in s for k in ['credit', 'debit', 'card', 'cc']):
        return 'Card'
    
    # Cash on Delivery
    if 'cod' in s or 'cashondelivery' in s or 'cash' in s:
        return 'COD'
    
    # Netbanking
    if 'netbank' in s or 'netbanking' in s:
        return 'NetBanking'
    
    return s.capitalize()  # fallback clean format

# apply
df['payment_method_clean'] = df['payment_method'].apply(clean_payment_method)

# show top results
df['payment_method_clean'].value_counts().head(20)


payment_method_clean
COD           24962
Card           6607
NetBanking     1596
Name: count, dtype: int64

In [20]:
# STEP 4: Duplicate detection (clean version)
from IPython.display import display

print("Total rows:", len(df))

# -------------------------------
# 1) Exact duplicate transaction_id
# -------------------------------

dup_tid = df[df.duplicated(subset=['transaction_id'], keep=False)].sort_values('transaction_id')

num_dup_tid_rows = len(dup_tid)
num_dup_tid_ids = dup_tid['transaction_id'].nunique()

print("\n[Exact Duplicate transaction_id]")
print("Rows duplicated:", num_dup_tid_rows)
print("Distinct transaction_id duplicated:", num_dup_tid_ids)

if num_dup_tid_rows > 0:
    print("\nSample exact-duplicate rows (first 5):")
    display(dup_tid.head(5))

# -------------------------------
# 2) Potential duplicates
# (same customer, product, date, amount)
# -------------------------------

dup_cand = df[df.duplicated(
    subset=['customer_id','product_id','order_date_clean','final_amount_inr'],
    keep=False
)].sort_values(['customer_id','product_id','order_date_clean','final_amount_inr'])

num_dup_cand_rows = len(dup_cand)
num_dup_cand_groups = dup_cand[['customer_id','product_id','order_date_clean','final_amount_inr']].drop_duplicates().shape[0]

print("\n[Potential Duplicate Rows]")
print("Rows duplicated in candidate set:", num_dup_cand_rows)
print("Distinct duplicate groups:", num_dup_cand_groups)

if num_dup_cand_rows > 0:
    print("\nSample potential-duplicate rows (first 5):")
    display(dup_cand.head(5))

print("\nSuggested strategy:")
print("- Keep one row per transaction_id if exact duplicates exist.")
print("- For customer/product/date/amount duplicates, inspect quantity:")
print("  If quantity differs -> legitimate multi-item order.")
print("  If rows identical -> treat as true duplicates.")


Total rows: 33165

[Exact Duplicate transaction_id]
Rows duplicated: 0
Distinct transaction_id duplicated: 0

[Potential Duplicate Rows]
Rows duplicated in candidate set: 330
Distinct duplicate groups: 165

Sample potential-duplicate rows (first 5):


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,original_price_clean,is_prime_member_clean,is_festival_sale_clean,is_prime_eligible_clean,customer_rating_clean,customer_city_clean,category_clean,subcategory_clean,delivery_days_clean,payment_method_clean
1851,TXN_2015_00001852,2015-01-23,CUST_2015_00000037,PROD_000053,OnePlus OnePlus 2 32GB Black,Electronics,Smartphones,OnePlus,102649.14,19.3,...,102649.14,False,True,True,5.0,delhi,electronics,smartphones,5.0,COD
33077,TXN_2015_00001852_DUP,2015-01-23,CUST_2015_00000037,PROD_000053,OnePlus OnePlus 2 32GB Black,Electronics,Smartphones,OnePlus,102649.14,19.3,...,102649.14,False,True,True,5.0,delhi,electronics,smartphones,5.0,COD
11861,TXN_2015_00011862,23-05-2015,CUST_2015_00000173,PROD_001820,OnePlus Neckband Premium,Electronics,Audio,OnePlus,19375.09,13.4,...,19375.09,False,False,False,3.5,delhi,electronics,audio,4.0,COD
33063,TXN_2015_00011862_DUP,23-05-2015,CUST_2015_00000173,PROD_001820,OnePlus Neckband Premium,Electronics,Audio,OnePlus,19375.09,13.4,...,19375.09,False,False,False,3.5,delhi,electronics,audio,4.0,COD
18161,TXN_2015_00018162,2015-08-25,CUST_2015_00000265,PROD_000071,Xiaomi Redmi 2 32GB Black,Electronics,Smartphones,Xiaomi,44203.12,0.0,...,44203.12,False,False,True,4.5,kolkata,electronics,smartphones,4.0,Card



Suggested strategy:
- Keep one row per transaction_id if exact duplicates exist.
- For customer/product/date/amount duplicates, inspect quantity:
  If quantity differs -> legitimate multi-item order.
  If rows identical -> treat as true duplicates.


In [21]:
# SAFE DEDUPLICATION for potential duplicate groups
from IPython.display import display

keys = ['customer_id','product_id','order_date_clean','final_amount_inr']

# 1) Show top duplicate groups (for inspection)
dup_groups = df.groupby(keys).size().reset_index(name='group_size')
dup_groups = dup_groups[dup_groups['group_size'] > 1].sort_values('group_size', ascending=False)
print("Duplicate groups found (count):", len(dup_groups))
print("Top 5 duplicate groups (customer_id, product_id, date, amount, group_size):")
display(dup_groups.head(5))

# show sample rows from one of the top duplicate groups to inspect
if len(dup_groups):
    sample_key = tuple(dup_groups.head(1)[keys].iloc[0])
    print("\nSample rows for the top duplicate group:", sample_key)
    sample_rows = df[(df['customer_id']==sample_key[0]) &
                     (df['product_id']==sample_key[1]) &
                     (df['order_date_clean']==sample_key[2]) &
                     (df['final_amount_inr']==sample_key[3])]
    display(sample_rows)

# 2) Conservative dedup strategy:
# Keep one row per duplicate group: the row with highest 'quantity' (ties -> first)
dup_keys = dup_groups[keys].itertuples(index=False, name=None)

idxs_to_keep = []
idxs_all_in_dup_groups = []

for key in dup_keys:
    cust, prod, date, amt = key
    mask = (df['customer_id']==cust) & (df['product_id']==prod) & (df['order_date_clean']==date) & (df['final_amount_inr']==amt)
    group_idx = df[mask].index.tolist()
    idxs_all_in_dup_groups.extend(group_idx)
    # choose index with max quantity
    idx_max_qty = df.loc[group_idx, 'quantity'].idxmax()
    idxs_to_keep.append(idx_max_qty)

idxs_all_in_dup_groups = pd.Index(idxs_all_in_dup_groups)
idxs_to_keep = pd.Index(idxs_to_keep)

# compute rows that would be dropped (members of duplicate groups but not the chosen keepers)
rows_to_drop = idxs_all_in_dup_groups.difference(idxs_to_keep)
print("\nRows in duplicate groups (total):", len(idxs_all_in_dup_groups))
print("Rows to KEEP (one per group):", len(idxs_to_keep))
print("Rows that will be DROPPED:", len(rows_to_drop))

# 3) Create a copy and drop them (safe: we'll not overwrite df until we confirm)
df_dedup = df.drop(index=rows_to_drop).copy()
print("\nShape before dedup:", df.shape)
print("Shape after dedup (if applied):", df_dedup.shape)

# 4) Add a flag column to original df for traceability, then replace df if you want
df['in_dup_group'] = df.index.isin(idxs_all_in_dup_groups)
df['dup_keep'] = df.index.isin(idxs_to_keep)
df['dup_to_drop'] = df.index.isin(rows_to_drop)

print("\nSummary flags counts:")
print(df[['in_dup_group','dup_keep','dup_to_drop']].sum())

# show a few rows marked for drop
if len(rows_to_drop):
    print("\nExample rows that would be dropped (first 5):")
    display(df.loc[rows_to_drop].head(5))

# NOTE: we have not overwritten `df`. If you are happy and want to make the change permanent:
# run: df = df_dedup.copy()
#
# If you want to save now:
# df_dedup.to_csv(out_dir / "amazon_india_2015_clean_part3_dedup.csv", index=False)


Duplicate groups found (count): 165
Top 5 duplicate groups (customer_id, product_id, date, amount, group_size):


,customer_id,product_id,order_date_clean,final_amount_inr,group_size
101,CUST_2015_00000037,PROD_000053,2015-01-23,82840.60,2
418,CUST_2015_00000173,PROD_001820,2015-05-23,16779.59,2
666,CUST_2015_00000265,PROD_000071,2015-08-25,132609.36,2
865,CUST_2015_00000326,PROD_000036,2015-07-27,88872.68,2
1149,CUST_2015_00000433,PROD_000071,2015-09-08,40751.36,2



Sample rows for the top duplicate group: ('CUST_2015_00000037', 'PROD_000053', Timestamp('2015-01-23 00:00:00'), np.float64(82840.6))


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,original_price_clean,is_prime_member_clean,is_festival_sale_clean,is_prime_eligible_clean,customer_rating_clean,customer_city_clean,category_clean,subcategory_clean,delivery_days_clean,payment_method_clean
1851,TXN_2015_00001852,2015-01-23,CUST_2015_00000037,PROD_000053,OnePlus OnePlus 2 32GB Black,Electronics,Smartphones,OnePlus,102649.14,19.3,...,102649.14,False,True,True,5.0,delhi,electronics,smartphones,5.0,COD
33077,TXN_2015_00001852_DUP,2015-01-23,CUST_2015_00000037,PROD_000053,OnePlus OnePlus 2 32GB Black,Electronics,Smartphones,OnePlus,102649.14,19.3,...,102649.14,False,True,True,5.0,delhi,electronics,smartphones,5.0,COD



Rows in duplicate groups (total): 330
Rows to KEEP (one per group): 165
Rows that will be DROPPED: 165

Shape before dedup: (33165, 46)
Shape after dedup (if applied): (33000, 46)

Summary flags counts:
in_dup_group    330
dup_keep        165
dup_to_drop     165
dtype: int64

Example rows that would be dropped (first 5):


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,is_prime_eligible_clean,customer_rating_clean,customer_city_clean,category_clean,subcategory_clean,delivery_days_clean,payment_method_clean,in_dup_group,dup_keep,dup_to_drop
33000,TXN_2015_00016501_DUP,2015-07-01,CUST_2015_00010086,PROD_001781,Boat Earbuds Premium,Electronics,Audio,Boat,17174.29,28.81,...,True,4.0,pune,electronics,audio,3.0,COD,True,False,True
33001,TXN_2015_00014828_DUP,2015-07-14,CUST_2015_00003428,PROD_001719,Xiaomi iPad 4GB RAM Black,Electronics,Tablets,Xiaomi,29675.17,50.41,...,True,4.5,delhi ncr,electronics,tablets,5.0,Card,True,False,True
33002,TXN_2015_00020641_DUP,2015-09-05,CUST_2015_00011426,PROD_000033,Samsung Galaxy S6 Edge 32GB White,Electronics,Smartphones,Samsung,167952.46,0.00,...,True,4.0,chennai,electronics,smartphones,3.0,COD,True,False,True
33003,TXN_2015_00007274_DUP,2015-04-24,CUST_2015_00004028,PROD_000066,Xiaomi Mi 4i 64GB Black,Electronics,Smartphones,Xiaomi,31102.54,25.55,...,True,4.5,moradabad,electronics,smartphones,3.0,COD,True,False,True
33004,TXN_2015_00002272_DUP,2015-01-28,CUST_2015_00000924,PROD_000077,Xiaomi Redmi 2 32GB Blue,Electronics,Smartphones,Xiaomi,20175.12,0.00,...,True,5.0,vadodara,electronics,smartphones,4.0,COD,True,False,True


In [22]:
# APPLY DEDUPLICATION
df = df_dedup.copy()

print("New shape after applying dedup:", df.shape)


New shape after applying dedup: (33000, 46)


In [23]:
df.to_csv(out_dir / "amazon_india_2015_clean_dedup.csv", index=False)
print("Saved final deduped file to:", out_dir / "amazon_india_2015_clean_dedup.csv")


Saved final deduped file to: c:\Users\DELL\Desktop\amazon_project\data_cleaned\amazon_india_2015_clean_dedup.csv


In [24]:
# PRICE OUTLIER DETECTION (flagging)
import numpy as np

# ensure numeric column exists
df['original_price_clean'] = pd.to_numeric(df['original_price_clean'], errors='coerce')

# product median price (skip NaNs)
prod_median = df.groupby('product_id')['original_price_clean'].median().rename('prod_median_price')
df = df.merge(prod_median, left_on='product_id', right_index=True, how='left')

# Flags
df['price_missing'] = df['original_price_clean'].isna()
df['price_gt_median_ratio'] = df['original_price_clean'] / df['prod_median_price']
df['price_gt_median_ratio'] = df['price_gt_median_ratio'].replace([np.inf, -np.inf], np.nan)

# suspicious if > 30x median OR absolute extremely high
df['price_suspect'] = False
mask_num = df['original_price_clean'].notna() & df['prod_median_price'].notna()
df.loc[mask_num & (df['price_gt_median_ratio'] > 30), 'price_suspect'] = True
df.loc[mask_num & (df['original_price_clean'] > 1_000_000), 'price_suspect'] = True

# counts
total_rows = len(df)
num_price_missing = int(df['price_missing'].sum())
num_suspect = int(df['price_suspect'].sum())

print("Total rows:", total_rows)
print("Price missing (clean):", num_price_missing)
print("Suspect price rows (>30x product median or >1,000,000):", num_suspect)

# show distribution of ratio for insight (summary)
print("\nprice ratio summary (non-null):")
print(df.loc[mask_num, 'price_gt_median_ratio'].describe().to_string())

# show top 20 most extreme suspects
print("\nTop 20 suspect rows (highest ratio):")
display(df.loc[df['price_suspect']].sort_values('price_gt_median_ratio', ascending=False).head(20)[
    ['transaction_id','product_id','product_name','original_price_clean','prod_median_price','price_gt_median_ratio','quantity']
])


Total rows: 33000
Price missing (clean): 985
Suspect price rows (>30x product median or >1,000,000): 98

price ratio summary (non-null):
count    32015.000000
mean         1.276214
std          5.083824
min         -1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max        100.000000

Top 20 suspect rows (highest ratio):


,transaction_id,product_id,product_name,original_price_clean,prod_median_price,price_gt_median_ratio,quantity
22573,TXN_2015_00022574,PROD_001733,OnePlus Galaxy Tab 4GB RAM Silver,4623409.0,46234.09,100.0,1
13209,TXN_2015_00013210,PROD_000085,Motorola Moto G (3rd Gen) 16GB Black,2194726.0,21947.26,100.0,1
26705,TXN_2015_00026706,PROD_000036,Samsung Galaxy S6 Edge 16GB Gold,8887268.0,88872.68,100.0,1
1319,TXN_2015_00001320,PROD_000073,Xiaomi Redmi 2 16GB White,4376740.0,43767.40,100.0,1
1833,TXN_2015_00001834,PROD_001713,Lenovo Pad 8GB RAM Silver,6744491.0,67444.91,100.0,2
2320,TXN_2015_00002321,PROD_000071,Xiaomi Redmi 2 32GB Black,4420312.0,44203.12,100.0,1
2465,TXN_2015_00002466,PROD_000035,Samsung Galaxy S6 Edge 32GB Blue,11728933.0,117289.33,100.0,2
1478,TXN_2015_00001479,PROD_001983,Samsung OLED TV,3888419.0,38884.19,100.0,1
2984,TXN_2015_00002985,PROD_000092,Motorola Moto G (3rd Gen) 16GB Gold,1786925.0,17869.25,100.0,1
3681,TXN_2015_00003682,PROD_000070,Xiaomi Redmi 2 16GB Black,2953470.0,29534.70,100.0,1


In [25]:
# APPLY CONSERVATIVE PRICE CORRECTION
df['price_corrected'] = df['original_price_clean'].copy()

changed = 0
changed_by_100 = 0
changed_by_10 = 0

mask = df['price_suspect'] & df['original_price_clean'].notna() & df['prod_median_price'].notna()

for idx in df.loc[mask].index:
    orig = df.at[idx, 'original_price_clean']
    med = df.at[idx, 'prod_median_price']
    
    candidate100 = orig / 100.0
    candidate10 = orig / 10.0
    
    # prefer /100 if it becomes reasonably close to product median
    if med > 0 and abs(candidate100 / med) <= 10:
        df.at[idx, 'price_corrected'] = candidate100
        changed += 1
        changed_by_100 += 1
        continue
    
    # fallback: try /10
    if med > 0 and abs(candidate10 / med) <= 10:
        df.at[idx, 'price_corrected'] = candidate10
        changed += 1
        changed_by_10 += 1

print("Total suspect rows:", int(mask.sum()))
print("Total corrected rows:", changed)
print("  corrected by /100:", changed_by_100)
print("  corrected by /10 :", changed_by_10)

# show sample corrections
display(df.loc[mask].head(10)[['transaction_id','product_id','original_price_clean','price_corrected','prod_median_price','price_gt_median_ratio']])


Total suspect rows: 98
Total corrected rows: 98
  corrected by /100: 98
  corrected by /10 : 0


,transaction_id,product_id,original_price_clean,price_corrected,prod_median_price,price_gt_median_ratio
445,TXN_2015_00000446,PROD_001737,3901009.0,39010.09,39010.09,100.0
1194,TXN_2015_00001195,PROD_000080,2631710.0,26317.10,26317.10,100.0
1319,TXN_2015_00001320,PROD_000073,4376740.0,43767.40,43767.40,100.0
1478,TXN_2015_00001479,PROD_001983,3888419.0,38884.19,38884.19,100.0
1833,TXN_2015_00001834,PROD_001713,6744491.0,67444.91,67444.91,100.0
2320,TXN_2015_00002321,PROD_000071,4420312.0,44203.12,44203.12,100.0
2465,TXN_2015_00002466,PROD_000035,11728933.0,117289.33,117289.33,100.0
2516,TXN_2015_00002517,PROD_001737,3901009.0,39010.09,39010.09,100.0
2984,TXN_2015_00002985,PROD_000092,1786925.0,17869.25,17869.25,100.0
3681,TXN_2015_00003682,PROD_000070,2953470.0,29534.70,29534.70,100.0


In [26]:
# STEP 7A: Convert delivery_charges to numeric
df['delivery_charges_clean'] = pd.to_numeric(df['delivery_charges'], errors='coerce')

print("Total missing delivery charges:", df['delivery_charges_clean'].isna().sum())
print("\nSummary stats:")
print(df['delivery_charges_clean'].describe())


Total missing delivery charges: 2640

Summary stats:
count    30360.0
mean         0.0
std          0.0
min          0.0
25%          0.0
50%          0.0
75%          0.0
max          0.0
Name: delivery_charges_clean, dtype: float64


In [27]:
# STEP 7B: Fill missing delivery charges with 0
df['delivery_charges_clean'] = df['delivery_charges_clean'].fillna(0)

print("Missing after fill:", df['delivery_charges_clean'].isna().sum())
print("Unique values:", df['delivery_charges_clean'].unique()[:10])


Missing after fill: 0
Unique values: [0.]


In [28]:
df.to_csv(out_dir / "amazon_india_2015_clean_final.csv", index=False)
print("Saved FULLY CLEANED file:", out_dir / "amazon_india_2015_clean_final.csv")


Saved FULLY CLEANED file: c:\Users\DELL\Desktop\amazon_project\data_cleaned\amazon_india_2015_clean_final.csv


In [9]:
def clean_orders_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Take a raw orders DataFrame and return a fully cleaned version.
    This uses our standard cleaning steps (Q1–Q5).
    """
    df = df.copy()

    # Q1: standardize order_date
    df = standardize_order_date(df, col="order_date")

    # Q2: clean original_price_inr
    df = clean_price_column(df, col="original_price_inr")

    # Q3: clean customer_rating
    df = clean_customer_rating(df, col="customer_rating")

    # Q4: standardize customer city/state
    df = standardize_location(
        df,
        city_col="customer_city",
        state_col="customer_state",
    )

    # Q5: normalize boolean-like columns
    boolean_cols = [
        "is_prime_member",
        "is_festival_sale",
        "is_prime_eligible",
    ]
    df = normalize_boolean_columns(df, boolean_cols)

    return df



In [2]:
def clean_orders_df(df: pd.DataFrame) -> pd.DataFrame:
    # 👉 your 2015 cleaning steps are inside here
    return df


In [3]:
from pathlib import Path  # if you already imported this above, it's okay

# Our folders (from the notebook, go one level up ".." and then into data_raw / data_cleaned)
RAW_DIR = Path("../data_raw")
CLEAN_DIR = Path("../data_cleaned")

# Make sure the clean folder exists
CLEAN_DIR.mkdir(exist_ok=True)


In [4]:
def process_one_file(raw_path: Path):
    """
    Read one raw CSV, clean it using clean_orders_df, 
    save the cleaned file, and print what happened.
    """
    print(f"\n----- Processing: {raw_path.name} -----")
    try:
        # 1) Read raw file
        df_raw = pd.read_csv(raw_path)

        # 2) Clean with your function from Step 1
        df_clean = clean_orders_df(df_raw)

        # 3) Build output file name: e.g. orders_2015.csv -> orders_2015_cleaned.csv
        clean_name = raw_path.stem + "_cleaned.csv"
        out_path = CLEAN_DIR / clean_name

        # 4) Save cleaned data
        df_clean.to_csv(out_path, index=False)

        print(f"✅ Saved cleaned file: {out_path}")
        return True, None   # success

    except Exception as e:
        print(f"❌ ERROR while processing {raw_path.name}: {e}")
        return False, str(e)   # failed, return error message


In [5]:
log_rows = []

# Go through every .csv file inside data_raw
for raw_path in sorted(RAW_DIR.glob("*.csv")):
    ok, err = process_one_file(raw_path)
    log_rows.append({
        "file": raw_path.name,
        "success": ok,
        "error": err
    })

# Turn the log into a DataFrame
log_df = pd.DataFrame(log_rows)

# Save the log into data_cleaned folder
log_path = CLEAN_DIR / "cleaning_log.csv"
log_df.to_csv(log_path, index=False)

print("\n====== Cleaning Summary ======")
print(log_df)
print(f"\n📝 Log saved to: {log_path}")



----- Processing: amazon_india_2015.csv -----
✅ Saved cleaned file: ..\data_cleaned\amazon_india_2015_cleaned.csv

----- Processing: amazon_india_2016.csv -----
✅ Saved cleaned file: ..\data_cleaned\amazon_india_2016_cleaned.csv

----- Processing: amazon_india_2017.csv -----
✅ Saved cleaned file: ..\data_cleaned\amazon_india_2017_cleaned.csv

----- Processing: amazon_india_2018.csv -----
✅ Saved cleaned file: ..\data_cleaned\amazon_india_2018_cleaned.csv

----- Processing: amazon_india_2019.csv -----
✅ Saved cleaned file: ..\data_cleaned\amazon_india_2019_cleaned.csv

----- Processing: amazon_india_2020.csv -----
✅ Saved cleaned file: ..\data_cleaned\amazon_india_2020_cleaned.csv

----- Processing: amazon_india_2021.csv -----
✅ Saved cleaned file: ..\data_cleaned\amazon_india_2021_cleaned.csv

----- Processing: amazon_india_2022.csv -----
✅ Saved cleaned file: ..\data_cleaned\amazon_india_2022_cleaned.csv

----- Processing: amazon_india_2023.csv -----
✅ Saved cleaned file: ..\data_cle

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path



### Define Cleaning Functions (Q1–Q5)


In [28]:
def clean_orders_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Take a raw orders DataFrame and return a fully cleaned version.
    This now includes Q1–Q10:
    - Dates, prices, ratings, locations, booleans
    - Category mapping, delivery_days cleaning
    - Duplicate handling, price outliers, payment method
    """
    df = df.copy()

    # Q1: dates
    df = standardize_order_date(df, col="order_date")

    # Q2: prices
    df = clean_price_column(df, col="original_price_inr")

    # Q3: ratings
    df = clean_customer_rating(df, col="customer_rating")

    # Q4: city/state – your actual column names
    df = standardize_location(
        df,
        city_col="customer_city",
        state_col="customer_state",
    )

    # Q5: booleans – your actual boolean-like columns
    boolean_cols = [
        "is_prime_member",
        "is_festival_sale",
        "is_prime_eligible",
    ]
    df = normalize_boolean_columns(df, boolean_cols)

    # Q6: category mapping
    df = clean_category(df, col="category")

    # Q7: delivery_days cleaning
    df = clean_delivery_days(df, col="delivery_days")

    # Q8: remove duplicate transaction_ids (just in case)
    if "transaction_id" in df.columns:
        before = len(df)
        df = df.drop_duplicates(subset="transaction_id")
        after = len(df)
        print(f"[Q8] Dropped {before - after} duplicate rows based on transaction_id.")

    # Q9: handle price outliers
    df = handle_price_outliers(df, col="original_price_inr")

    # Q10: payment_method standardization
    df = clean_payment_method(df, col="payment_method")

    return df




In [4]:
import os
from pathlib import Path

print("Current working folder:", os.getcwd())
print("Contents here:")
print(list(Path(".").iterdir()))


Current working folder: c:\Users\DELL\Desktop\amazon_project\notebooks
Contents here:
[WindowsPath('01_data_understanding.ipynb'), WindowsPath('02_data_cleaning_part1.ipynb'), WindowsPath('03_eda_overview.ipynb'), WindowsPath('04_final_report.html'), WindowsPath('04_final_report.ipynb')]


In [5]:
from pathlib import Path

data_raw_dir = Path("../data_raw")
for f in data_raw_dir.iterdir():
    print(f.name)


amazon_india_2015.csv
amazon_india_2016.csv
amazon_india_2017.csv
amazon_india_2018.csv
amazon_india_2019.csv
amazon_india_2020.csv
amazon_india_2021.csv
amazon_india_2022.csv
amazon_india_2023.csv
amazon_india_2024.csv
amazon_india_2025.csv


In [6]:
from pathlib import Path
import pandas as pd

raw_path = Path("../data_raw/amazon_india_2015.csv")  # using correct path
df_raw = pd.read_csv(raw_path)

df_raw.head()


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,is_festival_sale,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating
0,TXN_2015_00000001,2015-01-25,CUST_2015_00003884,PROD_000021,Samsung Galaxy S6 16GB Black,Electronics,Smartphones,Samsung,123614.29,27.91,...,True,Republic Day Sale,5.0,Delivered,1,2015,1,0.19,True,4.7
1,TXN_2015_00000002,2015-01-05,CUST_2015_00011709,PROD_000055,OnePlus OnePlus 2 16GB White,Electronics,Smartphones,OnePlus,54731.86,0.00,...,False,NaN,4.5,Delivered,1,2015,1,0.20,True,4.1
2,TXN_2015_00000003,2015-01-24,CUST_2015_00004782,PROD_000039,Samsung Galaxy Note 5 64GB Black,Electronics,Smartphones,Samsung,97644.25,46.93,...,True,Republic Day Sale,NaN,Delivered,1,2015,1,0.17,True,3.3
3,TXN_2015_00000004,2015-01-28,CUST_2015_00008105,PROD_000085,Motorola Moto G (3rd Gen) 16GB Black,Electronics,Smartphones,Motorola,"21,947.26",0.00,...,False,NaN,3.0,Delivered,1,2015,1,0.22,True,3.5
4,TXN_2015_00000005,2015-01-31,CUST_2015_00002955,PROD_000055,OnePlus OnePlus 2 16GB White,Electronics,Smartphones,OnePlus,54731.86,0.00,...,FALSE,NaN,4.0,Delivered,1,2015,1,0.20,True,4.1


In [7]:
df_clean = clean_orders(df_raw)
df_clean.head()


C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


,transaction_id,order_date,customer_id,product_id,product_name,category,subcategory,brand,original_price_inr,discount_percent,...,is_festival_sale,festival_name,customer_rating,return_status,order_month,order_year,order_quarter,product_weight_kg,is_prime_eligible,product_rating
0,TXN_2015_00000001,2015-01-25,CUST_2015_00003884,PROD_000021,Samsung Galaxy S6 16GB Black,Electronics,Smartphones,Samsung,123614.29,27.91,...,True,Republic Day Sale,5.0,Delivered,1,2015,1,0.19,True,4.7
1,TXN_2015_00000002,2015-05-01,CUST_2015_00011709,PROD_000055,OnePlus OnePlus 2 16GB White,Electronics,Smartphones,OnePlus,54731.86,0.00,...,False,NaN,4.5,Delivered,1,2015,1,0.20,True,4.1
2,TXN_2015_00000003,2015-01-24,CUST_2015_00004782,PROD_000039,Samsung Galaxy Note 5 64GB Black,Electronics,Smartphones,Samsung,97644.25,46.93,...,True,Republic Day Sale,NaN,Delivered,1,2015,1,0.17,True,3.3
3,TXN_2015_00000004,2015-01-28,CUST_2015_00008105,PROD_000085,Motorola Moto G (3rd Gen) 16GB Black,Electronics,Smartphones,Motorola,21947.26,0.00,...,False,NaN,3.0,Delivered,1,2015,1,0.22,True,3.5
4,TXN_2015_00000005,2015-01-31,CUST_2015_00002955,PROD_000055,OnePlus OnePlus 2 16GB White,Electronics,Smartphones,OnePlus,54731.86,0.00,...,FALSE,NaN,4.0,Delivered,1,2015,1,0.20,True,4.1


In [8]:
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33165 entries, 0 to 33164
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   transaction_id          33165 non-null  object        
 1   order_date              33165 non-null  datetime64[ns]
 2   customer_id             33165 non-null  object        
 3   product_id              33165 non-null  object        
 4   product_name            33165 non-null  object        
 5   category                33165 non-null  object        
 6   subcategory             33165 non-null  object        
 7   brand                   33165 non-null  object        
 8   original_price_inr      33165 non-null  Float64       
 9   discount_percent        33165 non-null  float64       
 10  discounted_price_inr    33165 non-null  float64       
 11  quantity                33165 non-null  int64         
 12  subtotal_inr            33165 non-null  float6

In [10]:
df_clean = clean_orders_df(df_raw)
df_clean.info()


C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33165 entries, 0 to 33164
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   transaction_id          33165 non-null  object        
 1   order_date              33165 non-null  datetime64[ns]
 2   customer_id             33165 non-null  object        
 3   product_id              33165 non-null  object        
 4   product_name            33165 non-null  object        
 5   category                33165 non-null  object        
 6   subcategory             33165 non-null  object        
 7   brand                   33165 non-null  object        
 8   original_price_inr      33165 non-null  Float64       
 9   discount_percent        33165 non-null  float64       
 10  discounted_price_inr    33165 non-null  float64       
 11  quantity                33165 non-null  int64         
 12  subtotal_inr            33165 non-null  float6

In [11]:
from pathlib import Path

out_path = Path("../data_cleaned/amazon_india_2015_cleaned.csv")
df_clean.to_csv(out_path, index=False)

print("Saved to:", out_path)


Saved to: ..\data_cleaned\amazon_india_2015_cleaned.csv


In [12]:
from pathlib import Path
Path("../data_cleaned_v2").mkdir(exist_ok=True)


In [13]:
from pathlib import Path
import pandas as pd

raw_dir = Path("../data_raw")
clean_dir = Path("../data_cleaned_v2")

for file in raw_dir.iterdir():
    if file.suffix == ".csv":
        print(f"Cleaning → {file.name}")
        
        # Load raw file
        df_raw = pd.read_csv(file)
        
        # Clean using our pipeline
        df_clean = clean_orders_df(df_raw)
        
        # Create output file name
        out_path = clean_dir / file.name.replace(".csv", "_cleaned_v2.csv")
        
        # Save cleaned file
        df_clean.to_csv(out_path, index=False)
        
        print(f"✔ Saved → {out_path}\n")


Cleaning → amazon_india_2015.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


✔ Saved → ..\data_cleaned_v2\amazon_india_2015_cleaned_v2.csv

Cleaning → amazon_india_2016.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


✔ Saved → ..\data_cleaned_v2\amazon_india_2016_cleaned_v2.csv

Cleaning → amazon_india_2017.csv


...

✔ Saved → ..\data_cleaned_v2\amazon_india_2023_cleaned_v2.csv

Cleaning → amazon_india_2024.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


✔ Saved → ..\data_cleaned_v2\amazon_india_2024_cleaned_v2.csv

Cleaning → amazon_india_2025.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


✔ Saved → ..\data_cleaned_v2\amazon_india_2025_cleaned_v2.csv



In [14]:
df_raw['category'].unique()[:50]


array(['Electronics', 'ELECTRONICS', 'Electronicss',
       'Electronics & Accessories', 'Electronic'], dtype=object)

In [15]:
category_map = {
    "electronics": "Electronics",
    "electronic": "Electronics",
    "electronicss": "Electronics",
    "electronics & accessories": "Electronics",
}


In [16]:
def clean_category(df: pd.DataFrame, col: str = "category") -> pd.DataFrame:
    if col not in df.columns:
        return df
    
    df = df.copy()
    
    # Convert to lowercase for easier matching
    s = df[col].astype("string").str.strip().str.lower()
    
    category_map = {
        "electronics": "Electronics",
        "electronic": "Electronics",
        "electronicss": "Electronics",
        "electronics & accessories": "Electronics",
    }
    
    df[col] = s.replace(category_map)
    
    # Convert back to title case formatting
    df[col] = df[col].str.title()
    
    return df


In [17]:
df_clean = clean_category(df_clean)
df_clean['category'].unique()


<StringArray>
['Electronics']
Length: 1, dtype: string

In [18]:
df_clean["delivery_days"].unique()[:50]


array(['3', '2', '1', '5', '6', '4', '0', '-1', '1-2 days', '7',
       'Same Day', 'Express', '15'], dtype=object)

In [19]:
def clean_delivery_days(df: pd.DataFrame, col: str = "delivery_days") -> pd.DataFrame:
    """
    Clean delivery_days column and convert to numeric days.
    Examples:
      "3" -> 3
      "1-2 days" -> 2
      "Same Day" -> 0
      "Express" -> 1
      "-1" -> NaN
    """
    if col not in df.columns:
        return df

    df = df.copy()
    s = df[col].astype("string").str.strip()

    def parse_delivery(val):
        if val is None or pd.isna(val) or val == "":
            return pd.NA

        v = val.lower().strip()

        # Same day / express keywords
        if v in ("same day", "sameday"):
            return 0
        if v == "express":
            return 1  # assume delivered within 1 day

        # Remove the word "days" / "day"
        v = v.replace("days", "").replace("day", "").strip()

        # Handle ranges like "1-2"
        if "-" in v:
            parts = v.split("-")
            nums = []
            for p in parts:
                p = p.strip()
                try:
                    nums.append(float(p))
                except:
                    pass
            if nums:
                # take the max in the range, e.g. 1-2 -> 2
                val_num = max(nums)
                if val_num >= 0:
                    return val_num
                else:
                    return pd.NA

        # Handle simple numbers like "3", "0", "-1"
        try:
            n = float(v)
            if n < 0:
                return pd.NA
            return n
        except:
            return pd.NA

    df[col] = s.map(parse_delivery).astype("Float64")

    return df


In [20]:
df_clean = clean_delivery_days(df_clean, "delivery_days")
df_clean["delivery_days"].unique(), df_clean["delivery_days"].dtype


(<FloatingArray>
 [3.0, 2.0, 1.0, 5.0, 6.0, 4.0, 0.0, 7.0, 15.0]
 Length: 9, dtype: Float64,
 Float64Dtype())

In [21]:
df_clean.duplicated(subset="transaction_id").sum()


np.int64(0)

In [22]:
df_clean["original_price_inr"].describe()


count          77385.0
mean       60163.93598
std      311401.670915
min         -239246.47
25%           21351.47
50%           31831.49
75%           63597.67
max         23751352.0
Name: original_price_inr, dtype: Float64

In [23]:
def handle_price_outliers(
    df: pd.DataFrame,
    col: str = "original_price_inr",
    lower_bound: float = 0.0,
    upper_quantile: float = 0.99,
) -> pd.DataFrame:
    """
    Handle extreme values in price column.
    - Any value <= lower_bound becomes NaN
    - Any very high value is capped at the given upper quantile
    """
    if col not in df.columns:
        return df

    df = df.copy()

    s = df[col].astype("Float64")

    # 1) Remove clearly invalid (negative / zero if we want)
    s = s.mask(s <= lower_bound, other=pd.NA)

    # 2) Compute upper cap from remaining positive values
    positive = s.dropna()
    if len(positive) == 0:
        df[col] = s
        return df

    cap = positive.quantile(upper_quantile)

    # 3) Clip prices above cap
    s = s.clip(upper=cap)

    df[col] = s

    print(f"[{col}] Applied outlier handling: <= {lower_bound} set to NaN, values above {cap:.2f} capped.")

    return df


In [24]:
df_clean = handle_price_outliers(df_clean, "original_price_inr")
df_clean["original_price_inr"].describe()


[original_price_inr] Applied outlier handling: <= 0.0 set to NaN, values above 220917.55 capped.


count         77179.0
mean     48796.612505
std      40818.096885
min           1067.27
25%          21443.85
50%          31880.98
75%           63788.0
max         220917.55
Name: original_price_inr, dtype: Float64

In [25]:
def clean_payment_method(df: pd.DataFrame, col: str = "payment_method") -> pd.DataFrame:
    """
    Standardize payment_method values into a few categories:
    COD, Card, UPI, Net Banking, Wallet, EMI, Other
    """
    if col not in df.columns:
        return df

    df = df.copy()
    s = df[col].astype("string").str.strip().str.lower()

    def normalize(pm: str):
        if pm is None or pd.isna(pm) or pm == "":
            return "Other"

        text = pm.lower()

        # COD / Cash on Delivery
        if "cod" in text or "cash on delivery" in text or text == "cash":
            return "COD"

        # UPI (GPay, PhonePe, BHIM, etc.)
        if "upi" in text or "gpay" in text or "google pay" in text or "phonepe" in text or "bhim" in text:
            return "UPI"

        # Cards
        if "credit card" in text or "debit card" in text or "card" in text:
            return "Card"

        # Net Banking
        if "net banking" in text or "netbanking" in text or "internet banking" in text:
            return "Net Banking"

        # Wallets (Paytm wallet, Amazon Pay, etc.)
        if "wallet" in text or "amazon pay" in text or "paytm" in text:
            return "Wallet"

        # EMI
        if "emi" in text:
            return "EMI"

        return "Other"

    df[col] = s.map(normalize).astype("string")

    return df


In [26]:
df_clean = clean_payment_method(df_clean, "payment_method")
df_clean["payment_method"].value_counts()


payment_method
UPI            46539
Card           15511
COD             6236
Other           5266
Net Banking     2285
Wallet          1548
Name: count, dtype: Int64

In [29]:
from pathlib import Path
import pandas as pd

raw_path = Path("../data_raw/amazon_india_2015.csv")
df_raw = pd.read_csv(raw_path)

df_clean = clean_orders_df(df_raw)
df_clean.info()


C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_3936\1424662617.py:15: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


[Q8] Dropped 0 duplicate rows based on transaction_id.
[original_price_inr] Applied outlier handling: <= 0.0 set to NaN, values above 190469.10 capped.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33165 entries, 0 to 33164
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   transaction_id          33165 non-null  object        
 1   order_date              33165 non-null  datetime64[ns]
 2   customer_id             33165 non-null  object        
 3   product_id              33165 non-null  object        
 4   product_name            33165 non-null  object        
 5   category                33165 non-null  string        
 6   subcategory             33165 non-null  object        
 7   brand                   33165 non-null  object        
 8   original_price_inr      33081 non-null  Float64       
 9   discount_percent        33165 non-null  float64       
 10  discounted_pri

In [1]:
from pathlib import Path

clean_dir = Path("../data_cleaned_v3")
clean_dir.mkdir(exist_ok=True)

print("Folder created:", clean_dir.resolve())


Folder created: C:\Users\DELL\Desktop\amazon_project\data_cleaned_v3


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

# Q1: standardize order_date
def standardize_order_date(df: pd.DataFrame, col: str = "order_date") -> pd.DataFrame:
    if col not in df.columns:
        return df
    s = df[col].astype("string").str.strip()

    def parse_mixed_dates(x):
        if x is None or pd.isna(x) or x == "":
            return pd.NaT
        try:
            v = float(x)
            if v > 20000:
                return pd.to_datetime("1899-12-30") + pd.to_timedelta(int(v), unit="D")
        except Exception:
            pass
        return pd.to_datetime(x, errors="coerce", dayfirst=True)

    df[col] = s.map(parse_mixed_dates).astype("datetime64[ns]")
    return df


# Q2: clean original_price_inr
def clean_price_column(df: pd.DataFrame, col: str = "original_price_inr") -> pd.DataFrame:
    if col not in df.columns:
        return df
    s = df[col].astype("string").str.strip()
    s = (
        s.str.replace("₹", "", regex=False)
         .str.replace("Rs.", "", regex=False)
         .str.replace("Rs", "", regex=False)
         .str.replace(",", "", regex=False)
         .str.replace(" ", "", regex=False)
         .str.replace("--", "", regex=False)
    )
    s = s.replace({"free": "0", "Free": "0", "": np.nan})
    df[col] = pd.to_numeric(s, errors="coerce").astype("Float64")
    return df


# Q3: clean customer_rating
def clean_customer_rating(df: pd.DataFrame, col: str = "customer_rating") -> pd.DataFrame:
    if col not in df.columns:
        return df
    s = df[col].astype("string").str.strip()
    s_lower = s.str.lower()

    text_map = {
        "one": 1, "two": 2, "three": 3, "four": 4, "five": 5,
        "poor": 1, "average": 3, "good": 4, "excellent": 5
    }

    def parse_rating(x, x_lower):
        if x is None or pd.isna(x) or x == "":
            return np.nan
        if "⭐" in x:
            c = x.count("⭐")
            if 1 <= c <= 5:
                return float(c)
        if x_lower in text_map:
            return float(text_map[x_lower])
        try:
            v = float(x)
            if 0 <= v <= 5:
                return v
        except:
            pass
        return np.nan

    df[col] = [parse_rating(orig, low) for orig, low in zip(s, s_lower)]
    df[col] = df[col].astype("float")
    return df


# Q4: standardize city/state
def standardize_location(df: pd.DataFrame, city_col="customer_city", state_col="customer_state") -> pd.DataFrame:
    def clean_name(series):
        series = series.astype("string")
        series = series.str.strip()
        series = series.str.replace(r"\s+", " ", regex=True)
        series = series.str.title()
        return series

    if city_col in df.columns:
        df[city_col] = clean_name(df[city_col])

    if state_col in df.columns:
        df[state_col] = clean_name(df[state_col])
        state_map = {
            "Up": "Uttar Pradesh",
            "U.P.": "Uttar Pradesh",
            "Delhi": "Delhi",
            "Nct Of Delhi": "Delhi",
            "Mh": "Maharashtra",
        }
        df[state_col] = df[state_col].replace(state_map)

    return df


# Q5: normalize boolean-like columns
def normalize_boolean_columns(df: pd.DataFrame, cols):
    truthy = {"yes", "y", "true", "t", "1", "paid", "completed"}
    falsy = {"no", "n", "false", "f", "0", "unpaid", "pending", ""}

    df = df.copy()
    for col in cols:
        if col not in df.columns:
            continue
        s = df[col].astype("string").str.lower().str.strip()

        def to_bool(x):
            if x in truthy:
                return True
            if x in falsy:
                return False
            return pd.NA

        df[col] = s.map(to_bool).astype("boolean")
    return df


# Q6: category mapping
def clean_category(df: pd.DataFrame, col: str = "category") -> pd.DataFrame:
    if col not in df.columns:
        return df
    
    df = df.copy()
    s = df[col].astype("string").str.strip().str.lower()
    
    category_map = {
        "electronics": "Electronics",
        "electronic": "Electronics",
        "electronicss": "Electronics",
        "electronics & accessories": "Electronics",
    }
    
    df[col] = s.replace(category_map)
    df[col] = df[col].str.title()
    
    return df


# Q7: delivery_days cleaning
def clean_delivery_days(df: pd.DataFrame, col: str = "delivery_days") -> pd.DataFrame:
    if col not in df.columns:
        return df

    df = df.copy()
    s = df[col].astype("string").str.strip()

    def parse_delivery(val):
        if val is None or pd.isna(val) or val == "":
            return pd.NA

        v = val.lower().strip()

        if v in ("same day", "sameday"):
            return 0
        if v == "express":
            return 1

        v = v.replace("days", "").replace("day", "").strip()

        if "-" in v:
            parts = v.split("-")
            nums = []
            for p in parts:
                p = p.strip()
                try:
                    nums.append(float(p))
                except:
                    pass
            if nums:
                val_num = max(nums)
                if val_num >= 0:
                    return val_num
                else:
                    return pd.NA

        try:
            n = float(v)
            if n < 0:
                return pd.NA
            return n
        except:
            return pd.NA

    df[col] = s.map(parse_delivery).astype("Float64")
    return df


# Q9: handle price outliers
def handle_price_outliers(
    df: pd.DataFrame,
    col: str = "original_price_inr",
    lower_bound: float = 0.0,
    upper_quantile: float = 0.99,
) -> pd.DataFrame:
    if col not in df.columns:
        return df

    df = df.copy()
    s = df[col].astype("Float64")

    s = s.mask(s <= lower_bound, other=pd.NA)

    positive = s.dropna()
    if len(positive) == 0:
        df[col] = s
        return df

    cap = positive.quantile(upper_quantile)
    s = s.clip(upper=cap)

    df[col] = s

    print(f"[{col}] Applied outlier handling: <= {lower_bound} set to NaN, values above {cap:.2f} capped.")
    return df


# Q10: payment_method standardization
def clean_payment_method(df: pd.DataFrame, col: str = "payment_method") -> pd.DataFrame:
    if col not in df.columns:
        return df

    df = df.copy()
    s = df[col].astype("string").str.strip().str.lower()

    def normalize(pm: str):
        if pm is None or pd.isna(pm) or pm == "":
            return "Other"

        text = pm.lower()

        if "cod" in text or "cash on delivery" in text or text == "cash":
            return "COD"

        if "upi" in text or "gpay" in text or "google pay" in text or "phonepe" in text or "bhim" in text:
            return "UPI"

        if "credit card" in text or "debit card" in text or "card" in text:
            return "Card"

        if "net banking" in text or "netbanking" in text or "internet banking" in text:
            return "Net Banking"

        if "wallet" in text or "amazon pay" in text or "paytm" in text:
            return "Wallet"

        if "emi" in text:
            return "EMI"

        return "Other"

    df[col] = s.map(normalize).astype("string")
    return df


# MAIN PIPELINE: Q1–Q10
def clean_orders_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Take a raw orders DataFrame and return a fully cleaned version.
    Includes Q1–Q10.
    """
    df = df.copy()

    # Q1
    df = standardize_order_date(df, col="order_date")

    # Q2
    df = clean_price_column(df, col="original_price_inr")

    # Q3
    df = clean_customer_rating(df, col="customer_rating")

    # Q4
    df = standardize_location(
        df,
        city_col="customer_city",
        state_col="customer_state",
    )

    # Q5
    boolean_cols = [
        "is_prime_member",
        "is_festival_sale",
        "is_prime_eligible",
    ]
    df = normalize_boolean_columns(df, boolean_cols)

    # Q6
    df = clean_category(df, col="category")

    # Q7
    df = clean_delivery_days(df, col="delivery_days")

    # Q8
    if "transaction_id" in df.columns:
        before = len(df)
        df = df.drop_duplicates(subset="transaction_id")
        after = len(df)
        print(f"[Q8] Dropped {before - after} duplicate rows based on transaction_id.")

    # Q9
    df = handle_price_outliers(df, col="original_price_inr")

    # Q10
    df = clean_payment_method(df, col="payment_method")

    return df


In [3]:
from pathlib import Path
import pandas as pd

raw_dir = Path("../data_raw")
clean_dir = Path("../data_cleaned_v3")
clean_dir.mkdir(exist_ok=True)  # will create the folder if not there

for file in raw_dir.iterdir():
    if file.suffix == ".csv":
        print(f"Cleaning → {file.name}")
        
        df_raw = pd.read_csv(file)
        df_clean = clean_orders_df(df_raw)   # full Q1–Q10
        
        out_path = clean_dir / file.name.replace(".csv", "_cleaned_v3.csv")
        df_clean.to_csv(out_path, index=False)
        
        print(f"✔ Saved → {out_path}\n")


Cleaning → amazon_india_2015.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_12688\3600673031.py:20: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12688\3600673031.py:20: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


[Q8] Dropped 0 duplicate rows based on transaction_id.
[original_price_inr] Applied outlier handling: <= 0.0 set to NaN, values above 190469.10 capped.
✔ Saved → ..\data_cleaned_v3\amazon_india_2015_cleaned_v3.csv

Cleaning → amazon_india_2016.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_12688\3600673031.py:20: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12688\3600673031.py:20: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


[Q8] Dropped 0 duplicate rows based on transaction_id.
[original_price_inr] Applied outlier handling: <= 0.0 set to NaN, values above 197037.46 capped.
✔ Saved → ..\data_cleaned_v3\amazon_india_2016_cleaned_v3.csv

Cleaning → amazon_india_2017.csv


...

[Q8] Dropped 0 duplicate rows based on transaction_id.
[original_price_inr] Applied outlier handling: <= 0.0 set to NaN, values above 251224.59 capped.
✔ Saved → ..\data_cleaned_v3\amazon_india_2023_cleaned_v3.csv

Cleaning → amazon_india_2024.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_12688\3600673031.py:20: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12688\3600673031.py:20: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


[Q8] Dropped 0 duplicate rows based on transaction_id.
[original_price_inr] Applied outlier handling: <= 0.0 set to NaN, values above 245931.55 capped.
✔ Saved → ..\data_cleaned_v3\amazon_india_2024_cleaned_v3.csv

Cleaning → amazon_india_2025.csv


C:\Users\DELL\AppData\Local\Temp\ipykernel_12688\3600673031.py:20: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12688\3600673031.py:20: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(x, errors="coerce", dayfirst=True)


[Q8] Dropped 0 duplicate rows based on transaction_id.
[original_price_inr] Applied outlier handling: <= 0.0 set to NaN, values above 220917.55 capped.
✔ Saved → ..\data_cleaned_v3\amazon_india_2025_cleaned_v3.csv

